# **1. Dataset de Características**

In [5]:
import cv2
import glob
import os
import numpy as np
import tensorflow as tf

In [6]:
path='/kaggle/input/face-expression-recognition-dataset/images/validation/'

In [7]:
folders=os.listdir(path)

In [8]:
from tensorflow.keras.layers import Flatten
from tensorflow.keras.models import Sequential

In [9]:
base_model=tf.keras.applications.resnet.ResNet101(
            include_top=False,
            weights='imagenet',
            input_shape=(224,224,3)
)

In [17]:
model=Sequential()
model.add(base_model)
model.add(Flatten())

In [14]:
def get_convfeatures(path):
    img=cv2.imread(path)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img=cv2.resize(img,(224,224))
    return model.predict(np.array([img]))

In [15]:
features=[]
names=[]

In [18]:
for folder in folders:
    for fileName in glob.glob(path+folder+'/*.jpg'):
        names.append(fileName)
        features.append( get_convfeatures(fileName) )

In [19]:
features=np.array(features)
names=np.array(names) 

In [21]:
features = features[:,0,:]

# **2. K-means Clustering**

In [53]:
from nltk.cluster.kmeans import KMeansClusterer
from nltk.cluster.util import cosine_distance

In [54]:
kmeans=KMeansClusterer(7, distance=cosine_distance)

In [55]:
cluster = kmeans.cluster(features,assign_clusters=True)

In [56]:
cluster

In [57]:
kmeans.means()

In [58]:
group_images=[]
for centroid in kmeans.means():
    dist=[cosine_distance(centroid,feature) for feature in features]
    dist=np.array(dist)
    min_index=dist.argsort()[:5]
    group_images.append(names[min_index])

In [59]:
group_images

In [60]:
import matplotlib.pyplot as plt

In [61]:
plt.rcParams["figure.figsize"]=(10,10)
_,axarr=plt.subplots(len(group_images),len(group_images[0]))
for i, cluster in enumerate(group_images):
    for j,path_image in enumerate(cluster):
        img=cv2.imread(path_image)
        img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        axarr[i,j].imshow(img)